In [1]:
#!jupyter nbconvert --to=python Py_f2recom_toolbox.ipynb

In [ ]:
# plot mesh nodal area overview 
def plot_mesh_area(mesh, plot_globe = True, plot_poles=False, plot_zoom=False, levels=[]):
    '''check mesh setup nodal and area
    input: mesh object
    '''
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.cm as cm
    
    meshdiag = pf.get_meshdiag(mesh)
    
    if plot_globe:    
        # whole globe
        pf.plot(mesh,np.array(meshdiag.nod_area[0,:])/1e6,units='nodal area (km$^{2}$)', mapproj='rob',levels=levels, 
        cmap_extension='max', cmap = cm.viridis_r)
        
    if plot_poles:
        # ... and both poles
        pf.plot(mesh,np.array(meshdiag.nod_area[0,:])/1e6,units='nodal area (km$^{2}$)', mapproj='np', box=[-180, 180, 60, 90], cmap = cm.viridis_r,levels=levels)
        pf.plot(mesh,np.array(meshdiag.nod_area[0,:])/1e6,units='nodal area (km$^{2}$)', mapproj='sp', box=[-180, 180, -90, -60], cmap = cm.viridis_r,levels=levels)

    if plot_zoom: 
        # plot nodal area and mesh structure together
        pf.tplot(mesh, np.array(nod_area[0,:])/1e6, ptype='tri', box=[-30, 30, 60, 82], mapproj='np',lw=0.5,units='nodal area')
        pf.tplot(mesh, np.array(nod_area[0,:])/1e6, ptype='tri', box=[-30, 30, -30, 30], mapproj='merc',lw=0.5,units='nodal area')

In [ ]:
# plot mesh resolution overview 
def plot_mesh_resolution(mesh, plot_globe = True, plot_poles=False, plot_zoom=False, levels=[]):
    '''check mesh resolution
    input: mesh objects
    '''
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.cm as cm
    
    meshdiag = pf.get_meshdiag(mesh)
    resolution = np.array(2*np.sqrt(meshdiag.nod_area[0,:]/np.pi)/1000)
    
    if plot_globe:    
        # whole globe
        pf.plot(mesh,resolution,units='mesh resolution (km)', mapproj='rob',levels=levels, 
        cmap_extension='max', cmap = cm.viridis_r)
        
    if plot_poles:     
        # ... and both poles
        pf.plot(mesh,resolution,units='mesh resolution (km)', mapproj='np', box=[-180, 180, 60, 90], cmap = cm.viridis_r,levels=levels)
        pf.plot(mesh,resolution,units='mesh resolution (km)', mapproj='sp', box=[-180, 180, -90, -60], cmap = cm.viridis_r,levels=levels)

In [ ]:
def load_mat(filename):
    """
    This function should be called instead of direct scipy.io.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    """
    from scipy.io import loadmat, matlab
    import numpy as np
    def _check_vars(d):
        """
        Checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        """
        for key in d:
            if isinstance(d[key], matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
            elif isinstance(d[key], np.ndarray):
                d[key] = _toarray(d[key])
        return d

    def _todict(matobj):
        """
        A recursive function which constructs from matobjects nested dictionaries
        """
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _toarray(elem)
            else:
                d[strg] = elem
        return d

    def _toarray(ndarray):
        """
        A recursive function which constructs ndarray from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        """
        if ndarray.dtype != 'float64':
            elem_list = []
            for sub_elem in ndarray:
                if isinstance(sub_elem, matlab.mio5_params.mat_struct):
                    elem_list.append(_todict(sub_elem))
                elif isinstance(sub_elem, np.ndarray):
                    elem_list.append(_toarray(sub_elem))
                else:
                    elem_list.append(sub_elem)
            return np.array(elem_list)
        else:
            return ndarray

    data = loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_vars(data)

In [ ]:
def maredat_load(ncfileMaredat):
    '''
    Input: 
    ncfileMaredat
    
    Output:
    lon_maredat
    lat_maredat
    maredat_layered_sum: biomass in [mg C/m2], summed over depth ranges
    maredat_layered_mean: biomass in [mg C/m3], averaged over depth ranges
    '''
    
    import numpy as np
    from netCDF4 import Dataset
    
    f               = Dataset(ncfileMaredat, 'r')
    biomass_maredat = f['BIOMASS'][:] # mug C/L = mg C /m3
    lon_maredat     = f['LONGITUDE'][:]
    lat_maredat     = f['LATITUDE'][:]
    depth_maredat   = f['DEPTH'][:]

    biomass_maredat[:][biomass_maredat[:] == 1e+35] = np.nan # remove fill value
    #print(np.shape(biomass_maredat))

    # mean over year
    biomass_maredat_annual = np.nanmean(biomass_maredat[:],axis=0)
    #print(np.shape(biomass_maredat_annual))
    
    # create depth vector with original Maredat depths
    depths_vector = np.zeros(33) # shape: 33
    for k in np.arange(0,33):
        if k == 0:
            depths_vector[k] = (depth_maredat[k+1]-depth_maredat[k])/2
        elif k == 32:
            depths_vector[k] = (depth_maredat[k]-depth_maredat[k-1])/2
        else:
            depths_vector[k] = ((depth_maredat[k+1]-depth_maredat[k])/2) + ((depth_maredat[k]-depth_maredat[k-1])/2)

    depth = np.repeat(depths_vector[:, np.newaxis], 180, axis=1)
    depth = np.repeat(depth[:, :, np.newaxis], 360, axis=2) # shape: 33, 180, 360
    
    #print('depth_maredat: ', depth_maredat)
    #print('depths_vector: ', depths_vector)


    # depth indices
    dep_ind1  = np.nonzero((0 <= depth_maredat[:] )&(depth_maredat[:] <= 5))
    dep_ind2  = np.nonzero((5 < depth_maredat[:] )&(depth_maredat[:] <= 25))
    dep_ind3  = np.nonzero((25 < depth_maredat[:] )&(depth_maredat[:] <= 100))
    dep_ind4  = np.nonzero((100 < depth_maredat[:]))
    
    # multiply Maredat data with depth vector
    biomass_maredat_annual_aux = biomass_maredat_annual * depth
    
    #print('original Maredat data: ', biomass_maredat_annual[:,12,144])
    #print('Maredat data multiplied with depth: ', biomass_maredat_annual_aux[:,12,144])
    
    # sum over depth ranges
    biomass_maredat_0_5_sum      = np.squeeze(np.nansum(biomass_maredat_annual_aux[dep_ind1], axis=0))
    biomass_maredat_5_25_sum     = np.squeeze(np.nansum(biomass_maredat_annual_aux[dep_ind2], axis=0))
    biomass_maredat_25_100_sum   = np.squeeze(np.nansum(biomass_maredat_annual_aux[dep_ind3], axis=0))
    biomass_maredat_100_bot_sum  = np.squeeze(np.nansum(biomass_maredat_annual_aux[dep_ind4], axis=0))
    
    #print('Indices of second depth range: ', dep_ind2)
    #print('Maredat data summed over second depth range: ', biomass_maredat_5_25_sum[12,144])
    
    maredat_layered_sum = np.zeros((4,180,360))
    maredat_layered_sum[0,:,:] = biomass_maredat_0_5_sum
    maredat_layered_sum[1,:,:] = biomass_maredat_5_25_sum
    maredat_layered_sum[2,:,:] = biomass_maredat_25_100_sum
    maredat_layered_sum[3,:,:] = biomass_maredat_100_bot_sum
    
    # mean over depth ranges
    biomass_maredat_0_5_mean     = np.squeeze(np.nansum(biomass_maredat_annual_aux[dep_ind1],axis=0)/np.nansum(depth[dep_ind1],axis=0))
    biomass_maredat_5_25_mean    = np.squeeze(np.nansum(biomass_maredat_annual_aux[dep_ind2],axis=0)/np.nansum(depth[dep_ind2],axis=0))
    biomass_maredat_25_100_mean  = np.squeeze(np.nansum(biomass_maredat_annual_aux[dep_ind3],axis=0)/np.nansum(depth[dep_ind3],axis=0))
    biomass_maredat_100_bot_mean = np.squeeze(np.nansum(biomass_maredat_annual_aux[dep_ind4],axis=0)/np.nansum(depth[dep_ind4],axis=0))
    
    #print('Maredat data averaged over second depth range: ', biomass_maredat_5_25_mean[12,144])
    
    maredat_layered_mean = np.zeros((4,180,360))
    maredat_layered_mean[0,:,:] = biomass_maredat_0_5_mean
    maredat_layered_mean[1,:,:] = biomass_maredat_5_25_mean
    maredat_layered_mean[2,:,:] = biomass_maredat_25_100_mean
    maredat_layered_mean[3,:,:] = biomass_maredat_100_bot_mean
    
    
    return lat_maredat, lon_maredat, maredat_layered_sum, maredat_layered_mean

In [ ]:
def fesom_to_maredat_levels(resultpath, runid, years, mesh, lon_maredat, lat_maredat, FT):
    '''
    Process FESOM data analogue to Maredat layers 0-5, 5-25, 25-100, 100-bottom
    as means and sums
    and interpolate to 1x1 grid
    
    Input:
    resultpath as list
    runid as list
    years 
    mesh: mesh object as loaded with pyfesom2
    lon_maredat
    lat_maredat
    FT (functional type, i.e. Dia, Cocco, ...) 
    
    Output:
    fesom_layered_sum: array with FESOM on 1x1 grid, layered sums
    fesom_layered_mean: array with FESOM on 1x1 grid, layered means
    '''
    
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pylab as plt
    import cartopy.crs as ccrs
    
    # get interpolation input -> data of all levels
    # annual mean of fesom data already caluclated when reading model results
    #print('\nProcessing {0} for years {1}-{2}'.format(resultpath,years[0],years[-1]))
    
    biomass_fesom = pf.get_data(resultpath, FT, years, mesh, runid=runid, how="mean")
    biomass_fesom = biomass_fesom * 12.01
    #print('DiaC_fesom shape: ',np.shape(biomass_fesom))
    # shape DiaC_fesom: 126858, 47

    # interp preperation
    lons, lats = np.meshgrid(lon_maredat, lat_maredat)

    # load FESOM mesh diag 
    meshdiag=resultpath+'/'+runid+'.mesh.diag.nc'
    #!ncdump -h $meshdiag
    diag = pf.get_meshdiag(mesh,meshdiag=meshdiag,runid=runid)
    mesh_depths = -diag['nz'].values

    # because initially, depths are negative in fesom2, but we want the depth difference positive
    #print('mesh_depths: ',mesh_depths)
    #mesh_depths:  [-0.00e+00  5.00e+00  1.00e+01  2.00e+01  3.00e+01  4.00e+01  5.00e+01
      #6.00e+01  7.00e+01  8.00e+01  9.00e+01  1.00e+02  1.15e+02  1.35e+02
      #1.60e+02  1.90e+02  2.30e+02  2.80e+02  3.40e+02  4.10e+02  4.90e+02
      #5.80e+02  6.80e+02  7.90e+02  9.10e+02  1.04e+03  1.18e+03  1.33e+03
      #1.50e+03  1.70e+03  1.92e+03  2.15e+03  2.40e+03  2.65e+03  2.90e+03
      #3.15e+03  3.40e+03  3.65e+03  3.90e+03  4.15e+03  4.40e+03  4.65e+03
      #4.90e+03  5.15e+03  5.40e+03  5.65e+03  6.00e+03  6.25e+03]

    
    # interpolate fesom data to 1x1 grid
    biomass_fesom_interp = np.zeros((48, 180, 360))
    for d in range(0,len(mesh_depths)-1):
        biomass_fesom_interp[d,:] = pf.fesom2regular(
                data = biomass_fesom[:,d],
                mesh = mesh,
                lons = lons, 
                lats = lats)
       
    
    # create depth vector with original fesom depths
    depths_vector_fesom = np.zeros(48) # shape: 48
    for k in np.arange(0,48):
        if k == 0:
            depths_vector_fesom[k] = (mesh_depths[k+1]-mesh_depths[k])/2
        elif k == 47:
            depths_vector_fesom[k] = (mesh_depths[k]-mesh_depths[k-1])/2
        else:
            depths_vector_fesom[k] = ((mesh_depths[k+1]-mesh_depths[k])/2) + ((mesh_depths[k]-mesh_depths[k-1])/2)
    
    #print('depths_vector_fesom: ',depths_vector_fesom)
    #depths_vector_fesom:  [  2.5   5.    7.5  10.   10.   10.   10.   10.   10.   10.   10.   12.5
      #17.5  22.5  27.5  35.   45.   55.   65.   75.   85.   95.  105.  115.
     #125.  135.  145.  160.  185.  210.  225.  240.  125.  250.  250.  250.
     #250.  250.  250.  250.  250.  250.  250.  250.  250.  300.  300. ]
    
    depth_fesom = np.repeat(depths_vector_fesom[:, np.newaxis], 180, axis=1)
    depth_fesom = np.repeat(depth_fesom[:, :, np.newaxis], 360, axis=2) # shape: 48, 180, 360
    
    #print('mesh_depths: ', mesh_depths)
    #print('depths_vector_fesom: ', depths_vector_fesom)
    
    
    # depth indices similar to MarEDAT
    depth_range = [0,5,25,100,1e5]
    dep_ind1_fesom  = np.nonzero((0 <= mesh_depths[:] )&(mesh_depths[:] <= 5))
    dep_ind2_fesom  = np.nonzero((5 < mesh_depths[:] )&(mesh_depths[:] <= 25))
    dep_ind3_fesom  = np.nonzero((25 < mesh_depths[:] )&(mesh_depths[:] <= 100))
    dep_ind4_fesom  = np.nonzero((100 < mesh_depths[:]))


    # multiply fesom data with depth vector
    biomass_fesom_interp_aux = biomass_fesom_interp * depth_fesom
    
    #print('original fesom data: ', biomass_fesom_interp[:,12,144])
    #print('fesom data multiplied with depth: ', biomass_fesom_interp_aux[:,12,144])
    
    # sum over depth ranges
    biomass_fesom_0_5_sum      = np.squeeze(np.nansum(biomass_fesom_interp_aux[dep_ind1_fesom], axis=0))
    biomass_fesom_5_25_sum     = np.squeeze(np.nansum(biomass_fesom_interp_aux[dep_ind2_fesom], axis=0))
    biomass_fesom_25_100_sum   = np.squeeze(np.nansum(biomass_fesom_interp_aux[dep_ind3_fesom], axis=0))
    biomass_fesom_100_bot_sum  = np.squeeze(np.nansum(biomass_fesom_interp_aux[dep_ind4_fesom], axis=0))
    
    fesom_layered_sum = np.zeros((4,180,360))
    fesom_layered_sum[0,:,:] = biomass_fesom_0_5_sum
    fesom_layered_sum[1,:,:] = biomass_fesom_5_25_sum
    fesom_layered_sum[2,:,:] = biomass_fesom_25_100_sum
    fesom_layered_sum[3,:,:] = biomass_fesom_100_bot_sum
    
    #print('fesom_layered_sum: ', fesom_layered_sum[:,12,144])

    # mean over depth ranges
    biomass_fesom_0_5_mean     = np.squeeze(np.nansum(biomass_fesom_interp_aux[dep_ind1_fesom],axis=0)/np.nansum(depth_fesom[dep_ind1_fesom],axis=0))
    biomass_fesom_5_25_mean    = np.squeeze(np.nansum(biomass_fesom_interp_aux[dep_ind2_fesom],axis=0)/np.nansum(depth_fesom[dep_ind2_fesom],axis=0))
    biomass_fesom_25_100_mean  = np.squeeze(np.nansum(biomass_fesom_interp_aux[dep_ind3_fesom],axis=0)/np.nansum(depth_fesom[dep_ind3_fesom],axis=0))
    biomass_fesom_100_bot_mean = np.squeeze(np.nansum(biomass_fesom_interp_aux[dep_ind4_fesom],axis=0)/np.nansum(depth_fesom[dep_ind4_fesom],axis=0))
    
    #print('fesom data averaged over second depth range: ', biomass_fesom_5_25_mean[12,144])
    
    fesom_layered_mean = np.zeros((4,180,360))
    fesom_layered_mean[0,:,:] = biomass_fesom_0_5_mean
    fesom_layered_mean[1,:,:] = biomass_fesom_5_25_mean
    fesom_layered_mean[2,:,:] = biomass_fesom_25_100_mean
    fesom_layered_mean[3,:,:] = biomass_fesom_100_bot_mean
        
    
    return fesom_layered_sum, fesom_layered_mean
    

In [ ]:
def mask_model_with_maredat(fesom_data,maredat_data):
    '''
    Mask model data where no MarEDAT available, then fill masked_array for better processing
    
    Input:
    model: FESOM data array with 4 data layers, i.e. fesom_layered_sum as output of fesom_to_maredat_levels()
    maredat_layered_sum: maredat data as array[4] layered into 0-5, 5-25, 25-100, 100m-bottom
    
    '''
    
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pylab as plt
    import cartopy.crs as ccrs
    
    model_ma = fesom_data
    model_ma[0] = np.where(maredat_data[0]==0, np.nan, model_ma[0])
    model_ma[1] = np.where(maredat_data[1]==0, np.nan, model_ma[1])
    model_ma[2] = np.where(maredat_data[2]==0, np.nan, model_ma[2])
    model_ma[3] = np.where(maredat_data[3]==0, np.nan, model_ma[3])
    
    return model_ma
    

In [ ]:
def barplots(fesom_layered_sum_masked, maredat_layered_sum, lon_maredat, lat_maredat, FT, savefig):
    
    '''
    Plot barcharts with total global biomass in each depth layer for Maredat and fesom
    
    Input:
    fesom_layered_sum_masked
    maredat_layered_sum
    lon_maredat
    lat_maredat
    FT (functional group, e.g., DiaC, CoccoC, ...)
    savefig: save figures or not
    '''
    
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pylab as plt
    import cartopy.crs as ccrs
    import math 

    # create an area array to be able to integrate biomass spatially
    radius_earth = 6371000 # in m
    dx = [2*np.pi*radius_earth*np.cos(math.radians(i))/(360) for i in lat_maredat]
    dy = 2*np.pi*radius_earth/(360)
    dy = np.repeat(dy,len(lon_maredat))
    dx_all, dy_all = np.meshgrid(dx,dy)
    area_reg = np.transpose(dx_all*dy_all) # shape: 180, 360
    #area = np.repeat(area_reg[np.newaxis, :, :], 33, axis=0) # shape: 33, 180, 360
        
    # area correction
    fesom_layered_sum_masked_corr = np.zeros((4,180,360))
    fesom_layered_sum_masked_corr[0] = fesom_layered_sum_masked[0] * area_reg
    fesom_layered_sum_masked_corr[1] = fesom_layered_sum_masked[1] * area_reg
    fesom_layered_sum_masked_corr[2] = fesom_layered_sum_masked[2] * area_reg
    fesom_layered_sum_masked_corr[3] = fesom_layered_sum_masked[3] * area_reg
    
    maredat_layered_sum_corr = np.zeros((4,180,360))
    maredat_layered_sum_corr[0] = maredat_layered_sum[0] * area_reg
    maredat_layered_sum_corr[1] = maredat_layered_sum[1] * area_reg
    maredat_layered_sum_corr[2] = maredat_layered_sum[2] * area_reg
    maredat_layered_sum_corr[3] = maredat_layered_sum[3] * area_reg
    
    # compute sum
    fesom_latsum = np.zeros((4,360))
    fesom_totalsum = np.zeros((4))
    for d in range(0,4):
        fesom_latsum[d]   = np.nansum(fesom_layered_sum_masked_corr[d], axis=0)
        fesom_totalsum[d] = np.nansum(fesom_latsum[d], axis=0)
        fesom_totalsum[d] = fesom_totalsum[d]/1e15 # Tg
        
    maredat_latsum = np.zeros((4,360))
    maredat_totalsum = np.zeros((4))
    for d in range(0,4):
        maredat_latsum[d]   = np.nansum(maredat_layered_sum_corr[d], axis=0)
        maredat_totalsum[d] = np.nansum(maredat_latsum[d], axis=0)
        maredat_totalsum[d] = maredat_totalsum[d]/1e15 # Tg
        
        
    #print('fesom totalsum depth 1: ', fesom_totalsum[0])
    #print('fesom totalsum depth 2: ', fesom_totalsum[1])
    #print('fesom totalsum depth 3: ', fesom_totalsum[2])
    #print('fesom totalsum depth 4: ', fesom_totalsum[3])
    
    #print('maredat totalsum depth 1: ', maredat_totalsum[0])
    #print('maredat totalsum depth 2: ', maredat_totalsum[1])
    #print('maredat totalsum depth 3: ', maredat_totalsum[2])
    #print('maredat totalsum depth 4: ', maredat_totalsum[3])
    
    
    # plot preparation
    barWidth = 0.25
    ra3 = np.arange(len(maredat_totalsum))
    rb3 = [x + barWidth for x in ra3]
    rc3 = [x + barWidth for x in rb3]
    rd3 = [x + barWidth for x in rc3]
    re3 = [x + barWidth for x in rd3]
    rf3 = [x + barWidth for x in re3]
    rg3 = [x + barWidth for x in rf3]
    rh3 = [x + barWidth for x in rg3]
    ri3 = [x + barWidth for x in rh3]

    # plot
    fig = plt.figure(num=250,figsize=(10,5),dpi=300, facecolor='w', edgecolor='k')
    plt.bar(ra3, maredat_totalsum, color='blue', width=barWidth, edgecolor='dimgrey', label='Maredat')
    plt.bar(rb3, fesom_totalsum, color='orange', width=barWidth, edgecolor='dimgrey', label='Model')
    #plt.xticks([])
    plt.xticks([f * 0.33 for f in range(0,10)], ['0-5m','','','5-25m','','','25-100m','','','100m to bottom'], fontsize=9, rotation=90)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True) # labels along the bottom edge are off
    #plt.ylim(ymax = 18e-3, ymin = 0)
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    plt.ylabel('Annual mean of globally integrated '+FT+' [Tg C]', fontsize=9)
    plt.legend(fontsize=8, loc='upper left', frameon=False)
    
    if(savefig == True):
        fig.savefig(savepath+'Barplot_MaredatFesom_'+FT+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
        
    plt.show()


    
    


In [ ]:
def plot_2cols(fesom_layered_sum, figlabel, savelabel, maredat_layered_sum, lon_maredat, lat_maredat, 
                      mapproj, years, savefig=False, savepath=''):
    '''
    Plot MarEDAT and FESOM data side by side in two columns, seperated by depth ranges
    
    Input:
    model: FESOM data array with 4 data layers, i.e. fesom_layered_sum as output of fesom_to_maredat_levels()
    figlabel: part of xlabel for plot
    savelabel: used to save the produced graph ('MarEDAT_FESOM_depths_'+label...)
    maredat_layered_sum: maredat data as array[4] layered into 0-5, 5-25, 25-100, 100m-bottom, sum over each depth layer
    lon_maredat
    lat_maredat
    mapproj: as ccrs format 
    savefig=False
    savepath
    '''
    
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pylab as plt
    import cartopy.crs as ccrs
    
    
    if(mapproj == 'pc'): mapproj = ccrs.PlateCarree()
    elif(mapproj == 'rob'): mapproj = ccrs.Robinson()
    else: raise ValueError('Map projecction not supported')
        
    if(savepath == ''):
        if(savefig == True):
            raise ValueError('Input for saving graph insufficient')
    
    print(figlabel)
    
    fig, axes = plt.subplots(4,2, 
                             subplot_kw=dict(projection=mapproj),
                             gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                             figsize=(30,45))
    #fig.tight_layout()

    # 0-5 m maredat ---------------------------------------------------------------------------------------------
    m1 = axes[0,0]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered_sum[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='auto', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m1)
    m1.set_title('Maredat 0-5 m', fontsize=20)

    # 0-5 m FESOM
    m1 = axes[0,1]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(fesom_layered_sum[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='auto', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m1)
    m1.set_title('FESOM 0-5 m', fontsize=20)

    # 5-25 m maredat ---------------------------------------------------------------------------------------------
    m2 = axes[1,0]
    f2 = m2.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered_sum[1]), 
                       transform = ccrs.PlateCarree(),
                        shading='auto', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m2)
    m2.set_title('Maredat 5-25 m', fontsize=20)

    # 5-25 m FESOM
    m2 = axes[1,1]
    f2 = m2.pcolormesh(lon_maredat, lat_maredat, np.log10(fesom_layered_sum[1]), 
                       transform = ccrs.PlateCarree(),
                        shading='auto', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m2)
    m2.set_title('FESOM 5-25 m', fontsize=20)

    # 25-100 m maredat ---------------------------------------------------------------------------------------------
    m3 = axes[2,0]
    f3 = m3.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered_sum[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='auto', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m3)
    m3.set_title('Maredat 25-100 m', fontsize=20)

    # 25-100 m FESOM
    m3 = axes[2,1]
    f3 = m3.pcolormesh(lon_maredat, lat_maredat, np.log10(fesom_layered_sum[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='auto', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m3)
    m3.set_title('FESOM 25-100 m', fontsize=20)

    # 100-bottom m maredat ---------------------------------------------------------------------------------------------
    m4 = axes[3,0]
    f4 = m4.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered_sum[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='auto', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m4)
    m4.set_title('Maredat 100 m - bottom', fontsize=20)

    # 100-bottom m
    m4 = axes[3,1]
    f4 = m4.pcolormesh(lon_maredat, lat_maredat, np.log10(fesom_layered_sum[3]), 
                       transform = ccrs.PlateCarree(),
                        shading='auto', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m4)
    m4.set_title('FESOM 100 m - bottom', fontsize=20)

    # ---------------------------------------------------------------------------------------------
    # add one colorbar for all plots below figure
    cbar = fig.colorbar(f1,
                        ax = axes[3,:2], 
                        location ='bottom',
                        ticks=[np.log10(0.001),np.log10(0.01), np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(10),np.log10(50),np.log10(100)],
                        fraction=0.1, pad=0.1) 
    cbar.ax.set_xticklabels(['0.001','0.01','0.1','0.5','1','10','50','100'], fontsize=20) 
    cbar.set_label(PF+'[mg C m$^{-2}$]', fontsize=20)
    #cbar.set_label('Log$_{10}$ Biomass [$\mu$g L $^{-1}$]', fontsize=20)

    if(savefig == True):
        fig.savefig(savepath+'Maredat_FESOM_maps_'+FT+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
    plt.show()

In [ ]:
### define function
def plot_scatter(model_masked, figlabel, savelabel, maredat_layered_sum, 
                years, savefig=False, savepath='',
                corr = False):
    '''
    Plot MarEDAT ~ FESOM scattered in two columns (overview & focus), 
    seperated by depth ranges
    
    Input:
    model_masked: FESOM data array with 4 data layers
    figlabel: used for xlabel
    savelabel: used to save the produced graph ('MarEDAT_scatter_'+savelabel...)
    maredat_layered_sum: maredat data as array[4] layered into 0-5, 5-25, 25-100, 100m-bottom; sum in each depth layer
    savefig=False
    savepath
    corr: array(4,2) containing correlation coefficients (out of plot_Taylor)
    '''
    
    import numpy as np
    import matplotlib.pylab as plt
        
    if(savepath == ''):
        if(savefig == True):
            raise ValueError('Input for saving graph insufficient')
            
    if(corr != False):
        corr0 = corr[0][1]
        corr1 = corr[1][1]
        corr2 = corr[2][1]
        corr3 = corr[3][1]
    
    print(figlabel)
    
    fig, axes = plt.subplots(4,2, #gridspec_kw={'hspace': 0.001, 'wspace': 0.1}, 
                             figsize=(10,10))

    m1 = axes[0,0]
    m1.plot(np.log10(model_masked[0]),np.log10(maredat_layered_sum[0]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-2}$]')
    if(corr != False):
        m1.text(-2.5,4, "Depth range: 0-5 m, ccoef = {0:5.4f}".format(corr0),color='darkred')
    else: m1.text(-2.5,4, "Depth range: 0-5 m",color='darkred')
    

    m1 = axes[0,1]
    m1.plot(np.log10(model_masked[0]),np.log10(maredat_layered_sum[0]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(1,2)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(1.1,4, "Depth range: 0-5 m",color='darkred')

    # ---------------------------------------------------------------------------------------------------

    m1 = axes[1,0]
    m1.plot(np.log10(model_masked[1]),np.log10(maredat_layered_sum[1]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-2}$]')
    if(corr != False):
        m1.text(-2.5,4, "Depth range: 5-25 m, ccoef = {0:5.4f}".format(corr1),color='darkred')
    else: m1.text(-2.5,4, "Depth range: 5-25 m",color='darkred')

    m1 = axes[1,1]
    m1.plot(np.log10(model_masked[1]),np.log10(maredat_layered_sum[1]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(1,2)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(1.1,4, "Depth range: 5-25 m",color='darkred')

    # ---------------------------------------------------------------------------------------------------

    m1 = axes[2,0]
    m1.plot(np.log10(model_masked[2]),np.log10(maredat_layered_sum[2]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-2}$]')
    if(corr != False):
        m1.text(-2.5,4, "Depth range: 25-100 m, ccoef = {0:5.4f}".format(corr2),color='darkred')
    else: m1.text(-2.5,4, "Depth range: 25-100 m",color='darkred')

    m1 = axes[2,1]
    m1.plot(np.log10(model_masked[2]),np.log10(maredat_layered_sum[2]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(0,1.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(0.15,4, "Depth range: 25-100 m",color='darkred')

    # ---------------------------------------------------------------------------------------------------

    m1 = axes[3,0]
    m1.plot(np.log10(model_masked[3]),np.log10(maredat_layered_sum[3]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-2}$]')
    m1.set_xlabel('Model "'+figlabel+'" Log$_{10}$ biomass \n [mg C m$^{-2}$]')
    if(corr != False):
        m1.text(-2.5,4, "Depth range: 100 m - bot, ccoef = {0:5.4f}".format(corr3),color='darkred')
    else: m1.text(-2.5,4, "Depth range: 100 m - bot",color='darkred')

    m1 = axes[3,1]
    m1.plot(np.log10(model_masked[3]),np.log10(maredat_layered_sum[3]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-2,0.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.set_xlabel('Model '+figlabel+' Log$_{10}$ biomass \n [mg C m$^{-2}$]')
    m1.text(-1.7,4, "Depth range: 100 m - bottom",color='darkred')
    
    if(savefig == True):
        fig.savefig(savepath+'MarEDAT_scatter_'+savelabel+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
    plt.show()

In [ ]:
def maredat_overview_plot(maredat_layered_mean,lon_maredat, lat_maredat,mapproj = 'rob'):
    '''
    Input: 
    maredat_layered_mean: maredat data as array[4] layered into 0-5, 5-25, 25-100, 100m-bottom (mean in each depth interval)
    lon_maredat
    lat_maredat
    mapproj = 'rob': mapprojection abbreviation as in pyfesom2
    
    Output:
    figure
    '''
    
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pylab as plt
    import cartopy.crs as ccrs
    
    if(mapproj == 'pc'): mapproj = ccrs.PlateCarree()
    elif(mapproj == 'rob'): mapproj = ccrs.Robinson()
    else: raise ValueError('Map projecction not supported')
    
    fig, axes = plt.subplots(2,2, 
                             subplot_kw=dict(projection=mapproj),
                             gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                             figsize=(20,15))
    fig.tight_layout()

    # 0-5 m
    m1 = axes[0,0]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered_mean[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(100), 
                        cmap=plt.cm.viridis)
    mygrid(m1)
    m1.set_title('MarEDAT 0-5 m', fontsize=20)


    # 5-25 m
    m2 = axes[0,1]
    f2 = m2.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered_mean[1]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(100), 
                        cmap=plt.cm.viridis)
    mygrid(m2)
    m2.set_title('MarEDAT 5-25 m', fontsize=20)

    # 25-100 m
    m3 = axes[1,0]
    f3 = m3.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered_mean[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(100), 
                        cmap=plt.cm.viridis)
    mygrid(m3)
    m3.set_title('MarEDAT 25-100 m', fontsize=20)

    # 100-bottom m
    m4 = axes[1,1]
    f4 = m4.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered_mean[3]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(100), 
                        cmap=plt.cm.viridis)
    mygrid(m4)
    m4.set_title('MarEDAT 100 m - bottom', fontsize=20)

    # add one colorbar for all plots below figure
    cbar = fig.colorbar(f1,
                        ax = axes[1,:2], 
                        location ='bottom',
                        ticks=[np.log10(0.001),np.log10(0.01), np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(10),np.log10(50)],
                        fraction=0.1, pad=0.1) 
    cbar.ax.set_xticklabels(['0.001','0.01','0.1','0.5','1','10','50'], fontsize=20) 
    cbar.set_label('Diatom Biomass [mg C m$^{-3}$]', fontsize=20)
    #cbar.set_label('Log$_{10}$ Biomass [$\mu$g L $^{-1}$]', fontsize=20)

    plt.show()
    return fig

In [ ]:
### plot FESOM overview
def fesom_plot_depths(model, figlabel, savelabel, lon_maredat, lat_maredat, 
                      mapproj, years, savefig=False, savepath=''):
    '''
    Plot log10 FESOM layered means
    
    Input:
    model: FESOM data array with 4 data layers, i.e. DiaC_interp_all as output of fesom_to_maredat_levels()
    figlabel: used for xlabel
    savelabel: used to save the produced graph ('FESOM_depths_asMarEDat_'+label...)
    lon_maredat
    lat_maredat
    mapproj: as ccrs format
    savefig=False
    savepath
    '''
    
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pylab as plt
    import cartopy.crs as ccrs
    
    if(mapproj == 'pc'): mapproj = ccrs.PlateCarree()
    elif(mapproj == 'rob'): mapproj = ccrs.Robinson()
    else: raise ValueError('Map projecction not supported')   
        
    if(savepath == ''):
        if(savefig == True):
            raise ValueError('Input for saving graph insufficient')
        
    print(figlabel)
    
    fig, axes = plt.subplots(2,2, 
                             subplot_kw=dict(projection=mapproj),
                             gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                             figsize=(20,15))
    fig.tight_layout()

    # 0-5 m
    m1 = axes[0,0]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(model[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m1)
    m1.set_title('FESOM 0-5 m', fontsize=20)


    # 5-25 m
    m2 = axes[0,1]
    f2 = m2.pcolormesh(lon_maredat, lat_maredat, np.log10(model[1]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m2)
    m2.set_title('FESOM 5-25 m', fontsize=20)

    # 25-100 m
    m3 = axes[1,0]
    f3 = m3.pcolormesh(lon_maredat, lat_maredat, np.log10(model[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m3)
    m3.set_title('FESOM 25-100 m', fontsize=20)

    # 100-bottom m
    m4 = axes[1,1]
    f4 = m4.pcolormesh(lon_maredat, lat_maredat, np.log10(model[3]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m4)
    m4.set_title('FESOM 100 m - bottom', fontsize=20)

    # add one colorbar for all plots below figure
    cbar = fig.colorbar(f1,
                        ax = axes[1,:2], 
                        location ='bottom',
                        ticks=[np.log10(0.001),np.log10(0.01), np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(10),np.log10(50),np.log10(100),np.log10(200)],
                        fraction=0.1, pad=0.1) 
    cbar.ax.set_xticklabels(['0.001','0.01','0.1','0.5','1','10','50','100','200'], fontsize=20) 
    cbar.set_label(figlabel+', Diatom Biomass [mg C m$^{-3}$]', fontsize=20)
    #cbar.set_label('Log$_{10}$ Biomass [$\mu$g L $^{-1}$]', fontsize=20)

    if(savefig == True):
        fig.savefig(savepath+'FESOM_depths_asMarEDat_'+savelabel+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
    plt.show()

In [ ]:
def plt_Taylor_norm(data_ref,data_pred,
                mask=True,
                title='Normalized Taylor Diagram',
                label=['Observation', 'Model'],
                loc='right',
                tickRMSangle = 135.0,
                verbose = True,
                plot = True):
    '''
    Reference: observation
    Predicted: model data
    
    careful: here used different order as in input in skill_metrics: sm.taylor_statistics(pred,ref)
    
    Use this function to get Taylor statistics and plot them as normalized to std(reference) = 1
    
    Root mean sqare error are centered, see sm.centered_rms_dev? for details
    
    Applied normalization: 
        std_normalized(ref) = std(ref) / std(ref)
        std_normalized(pred) = std(pred) / std(ref)
        ... similar for CRMSD
        
    Input:
    data_ref: array with references/observation data
    data_pred: array with model data
    mask: bolean if observation data should be masked with np.where(data_pred !=0)
    title: string for plot title
    label: list with 2 entries for label for 'Observation', 'Model'
    loc: string, positional argument for legend
    tickRMSangle: used for ticks of RMSE in Taylor plot
    verbose: print calculated Taylor statistics
    plot: plot Taylor diagram
    
    Output:
    fig, sdev, crmsd, ccoef
    
    Changed style of marker in skill_metrics/plot_pattern_diagram_markers.py
    '''
    import numpy as np
    import matplotlib.pyplot as plt
    import skill_metrics as sm
                    
    if mask == True:
        # get statistics only from ocean gridpoints (where model data != 0)
        ind_stat = np.where(data_pred != 0)
        taylor_stats1 = sm.taylor_statistics(data_pred[ind_stat],data_ref[ind_stat])
    else:
        taylor_stats1 = sm.taylor_statistics(data_pred,data_ref)


    # original
    sdev = np.array([taylor_stats1['sdev'][0], 
                     taylor_stats1['sdev'][1]])
    
    crmsd = np.array([taylor_stats1['crmsd'][0],
                      taylor_stats1['crmsd'][1]])
    
    ccoef = np.array([taylor_stats1['ccoef'][0], 
                      taylor_stats1['ccoef'][1]])

    if verbose:
        print('\nOriginal Taylor stats:\nSDEV pred: {0:6.5f}, ref: {1:6.5f}\nCRMS pred: {2:6.5f}, ref: {3:6.5f}\nCORRCOEF: {4:6.5f}'.format(
            sdev[1],sdev[0],crmsd[1],crmsd[0],ccoef[1], ccoef[0]))
    
    # normalized    
    sdev = np.array([taylor_stats1['sdev'][0]/taylor_stats1['sdev'][0], 
                     taylor_stats1['sdev'][1]/taylor_stats1['sdev'][0]])
    
    crmsd = np.array([taylor_stats1['crmsd'][0]/taylor_stats1['sdev'][0],
                      taylor_stats1['crmsd'][1]/taylor_stats1['sdev'][0]])
    
    ccoef = np.array([taylor_stats1['ccoef'][0], 
                      taylor_stats1['ccoef'][1]])

    if verbose:
        print('\nNormalized Taylor stats:\nSDEV pred: {0:6.5f}, ref: {1:6.5f}\nCRMS pred: {2:6.5f}, ref: {3:6.5f}\nCORRCOEF: {4:6.5f}'.format(
            sdev[1],sdev[0],crmsd[1],crmsd[0],ccoef[1], ccoef[0]))

    if plot:
        if(sdev[1] > 1):
            axismax = np.round(sdev[1],0) +0.5
            print("Adjust axis max to {0}".format(axismax))
            tickSTD = np.arange(0,axismax,0.25)
        else: 
            axismax = 1.25
            tickSTD = [0,0.25,0.5,0.75,1.25]
    
        fig = plt.figure(figsize=(10,10), facecolor='w', edgecolor='k')
        sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                              titleOBS = 'observation', markerLabel = label,
                              markerLabelColor = 'c',
                              markerColor = 'c', markerLegend = 'on',
                              tickRMS = [0,0.25,0.5,0.75,1.25],
                              tickRMSangle = tickRMSangle,
                              colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                              titleRMS = 'off', tickSTD = tickSTD,# tickSTD = [0,0.25,0.5,0.75,1.25],
                              axismax = axismax, #axismax = 1.25, 
                              colSTD = 'b', styleSTD = '-.',
                              widthSTD = 1.0, titleSTD = 'on',
                              colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                              titleCOR = 'on')

        plt.title(title, loc=loc)   
    
    else:
        fig = 'none'

    return fig, sdev, crmsd, ccoef

In [ ]:
def plt_Taylor_comp(data_ref,data_pred,mesh,
                depth_array = (0,50,200,1000,2000,4000),
                mask=True,
                title='Normalized Taylor Diagram',
                label=['Observation'],
                loc='left',
                tickRMSangle = 135.0,
                verbose = True,
                plot = True):
    '''
    Reference: observation
    Predicted: model data
    
    careful: here used different order as in input in skill_metrics: sm.taylor_statistics(pred,ref)
    
    Use this function to get Taylor statistics and plot them as normalized to std(reference) = 1
    
    Root mean sqare error are centered, see sm.centered_rms_dev? for details
    
    Applied normalization: 
        std_normalized(ref) = std(ref) / std(ref)
        std_normalized(pred) = std(pred) / std(ref)
        ... similar for CRMSD
        
    Input:
    data_ref: array with references/observation data
    data_pred: array with model data
    mask: bolean if observation data should be masked with np.where(data_pred !=0)
    title: string for plot title
    label: list with 2 entries for label for 'Observation', 'Model'
    loc: string, positional argument for legend
    tickRMSangle: used for ticks of RMSE in Taylor plot
    verbose: print calculated Taylor statistics
    plot: plot Taylor diagram
    
    Output:
    fig, sdev, crmsd, ccoef
    
    Changed style of marker in skill_metrics/plot_pattern_diagram_markers.py
    '''
    import numpy as np
    import matplotlib.pyplot as plt
    import skill_metrics as sm
    import pyfesom2 as pf
    
    SDEV = []
    CRMSD = []
    CCOEF = []
    
    for d in depth_array:
        # get mesh index closest to desired depth
        i = pf.ind_for_depth(d,mesh) 
        # get midlevel depth
        plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)
        label.append('{0} m'.format(plot_depth))
        data_ref_temp = data_ref[:,i]
        data_pred_temp = data_pred[:,i]
        if mask == True:
            # get statistics only from ocean gridpoints (where model data != 0)
            ind_stat = np.where(data_pred_temp != 0)
            taylor_stats1 = sm.taylor_statistics(data_pred_temp[ind_stat],data_ref_temp[ind_stat])
        else:
            taylor_stats1 = sm.taylor_statistics(data_pred_temp,data_ref_temp)
        
        
        # normalized    
        sdev = np.array([taylor_stats1['sdev'][0]/taylor_stats1['sdev'][0], 
                         taylor_stats1['sdev'][1]/taylor_stats1['sdev'][0]])

        crmsd = np.array([taylor_stats1['crmsd'][0]/taylor_stats1['sdev'][0],
                          taylor_stats1['crmsd'][1]/taylor_stats1['sdev'][0]])

        ccoef = np.array([taylor_stats1['ccoef'][0], 
                          taylor_stats1['ccoef'][1]])

        if verbose:
            print('\nNormalized Taylor stats:\nSDEV pred: {0:6.5f}, ref: {1:6.5f}\nCRMS pred: {2:6.5f}, ref: {3:6.5f}\nCORRCOEF: {4:6.5f}'.format(
                sdev[1],sdev[0],crmsd[1],crmsd[0],ccoef[1], ccoef[0]))
            
        if d == depth_array[0]:
            SDEV = np.append(SDEV,sdev)
            CRMSD = np.append(CRMSD,crmsd)
            CCOEF = np.append(CCOEF,ccoef)
        else:
            SDEV = np.append(SDEV,sdev[1])
            CRMSD = np.append(CRMSD,crmsd[1])
            CCOEF = np.append(CCOEF,ccoef[1])
        
    if plot:
        if max(SDEV) > 1:
            axismax = np.round(max(SDEV),0) +0.5
            print("Adjust axis max to {0}".format(axismax))
            tickSTD = np.arange(0,axismax,0.25)
        else: 
            axismax = 1.25
            tickSTD = [0,0.25,0.5,0.75,1.25]
        
        fig = plt.figure(figsize=(10,10), facecolor='w', edgecolor='k')
        sm.taylor_diagram(SDEV,CRMSD,CCOEF, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                                  titleOBS = 'observation', markerLabel = label,
                                  markerLegend = 'on',
                                  markerColor = 'c',
                                  tickRMS = [0,0.25,0.5,0.75,1.25],
                                  tickRMSangle = tickRMSangle,
                                  colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                                  titleRMS = 'off', tickSTD = tickSTD,# tickSTD = [0,0.25,0.5,0.75,1.25],
                                  axismax = axismax, #axismax = 1.25, 
                                  colSTD = 'b', styleSTD = '-.',
                                  widthSTD = 1.0, titleSTD = 'on',
                                  colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                                  titleCOR = 'on')

        #plt.title(title, loc=loc) 
        print('displaying '+title)

    else:
        fig = 'none'

    return fig, sdev, crmsd, ccoef

In [1]:
def plot_Taylor(ref,pred,FT,savefig):
    
    '''
    Plot Taylor diagrams for all depth layers
    
    Input:
    ref: reference/observation data -> Maredat
    pred: predicted/model data -> Model
    FT -> functional group (e.g., DiaC, CoccoC, ...)
    savefig: save figures or not
    '''
    
    import skill_metrics as sm
    import numpy as np
    from Py_f2recom_toolbox import plt_Taylor_norm
    
    pred[np.isnan(pred)] = 0
    
    title_1 = FT+': 0-5 m'
    title_2 = FT+': 5-25 m'
    title_3 = FT+': 25-100 m'
    title_4 = FT+': 100m to bottom'
    
    fig_1, sdev_1, crmsd_1, ccoef_1 = plt_Taylor_norm(ref[0],pred[0],mask=True,title=title_1)
    fig_2, sdev_2, crmsd_2, ccoef_2 = plt_Taylor_norm(ref[1],pred[1],mask=True,title=title_2)
    fig_3, sdev_3, crmsd_3, ccoef_3 = plt_Taylor_norm(ref[2],pred[2],mask=True,title=title_3)
    fig_4, sdev_4, crmsd_4, ccoef_4 = plt_Taylor_norm(ref[3],pred[3],mask=True,title=title_4)
    
    if(savefig == True):
        fig_1.savefig(savepath+'Taylor_0-5m,'+FT+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
        fig_2.savefig(savepath+'Taylor_5-25m,'+FT+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
        fig_3.savefig(savepath+'Taylor_25-100m,'+FT+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
        fig_4.savefig(savepath+'Taylor_100-bottom,'+FT+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
    
    return fig_1, sdev_1, crmsd_1, ccoef_1, fig_2, sdev_2, crmsd_2, ccoef_2, fig_3, sdev_3, crmsd_3, ccoef_3, fig_4, sdev_4, crmsd_4, ccoef_4
    